In [2]:
import os

topic_file = os.getenv("TOPIC_PATH")
users_file = os.getenv("USERS_PATH")
ratings_file = os.getenv("RATINGS_PATH")

#topic_file = "/data/movie_data/movies.csv"
#users_file = "/data/movie_data/users.csv"
#ratings_file = "/data/movie_data/rating.csv"

In [3]:
import pandas as pd
topics = pd.read_csv(topic_file)

In [4]:
ratings = pd.read_csv(ratings_file)
ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [5]:
users = pd.read_csv(users_file)
users.head()

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [9]:
Mean= ratings.groupby(["movieId’], as_index = False, sort = False).\
    mean().rename(columns = {‘rating’: ‘rating_mean’})[[‘movieId’,’rating_mean’]]

SyntaxError: invalid character in identifier (<ipython-input-9-8ee75c7a7c5a>, line 1)